In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import optuna
import torch
from torch.autograd import Variable
import plotly
import tqdm as notebook_tqdm


In [2]:
x_train = pd.read_csv(
    "../data/processed/x_train_w_OHE.csv", index_col=0, dtype=str
)
x_test = pd.read_csv(
    "../data/processed/x_test_w_OHE.csv", index_col=0, dtype=str
)
y_train = pd.read_csv(
    "../data/processed/y_train.csv", index_col=0, dtype=float
).squeeze("columns").reset_index(drop=True)
y_test = pd.read_csv(
    "../data/processed/y_test.csv", index_col=0, dtype=float
).squeeze("columns").reset_index(drop=True)

x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42)


In [3]:
x_train.head()

,circuit_name,deenergize_time,restoration_time,key_communities,hftd_tier,total_affected,residential_affected,zip_code,longitude,latitude,...,zip_is_96035,zip_is_96051,zip_is_96055,zip_is_96059,zip_is_96069,zip_is_96073,zip_is_96076,zip_is_96080,zip_is_96096,zip_is_96137
452,SWIFT2110,2019-10-10 00:05:00,2019-10-11 14:17:00,"SAN JOSE, LIVERMORE",2.0,2232.0,2047.0,95148,-121.796959988177,37.3225680192999,...,0,0,0,0,0,0,0,0,0,0
443,STATION E EUREKA1105,2019-10-09 01:20:00,2019-10-09 23:10:00,EUREKA,0.0,1618.0,1264.0,95501,-124.180313467792,40.7934324220744,...,0,0,0,0,0,0,0,0,0,0
1804,SILVERADO,2021-08-17 18:22:00,2021-08-18 23:55:00,NAPA,3.0,1815.0,1516.0,94574,-122.459110675812,38.4998179385502,...,0,0,0,0,0,0,0,0,0,0
340,OREGON TRAIL1103,2019-10-09 01:39:00,2019-10-11 11:32:00,"REDDING, BELLA VISTA",2.0,1706.0,1599.0,96003,-122.322060242415,40.619045588007,...,0,0,0,0,0,0,0,0,0,0
1390,OREGON TRAIL,2020-10-22 03:23:00,2020-10-23 11:30:00,"PALO CEDRO, REDDING",2.0,952.0,843.0,96003,-122.322060242415,40.619045588007,...,0,0,0,0,0,0,0,0,0,0


In [4]:
zip_cols = x_train.columns[
    [re.search('zip_is', col) is not None for col in x_train.columns]
]


In [5]:
def get_correct_types_x(df, numeric_cols):
    for col in ['deenergize_time', 'restoration_time']:
        df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S')
    for col in numeric_cols:
        df[col] = df[col].astype(float)
    return df


numeric_cols = [
    'hftd_tier', 'total_affected', 'residential_affected',
    'longitude', 'latitude', 'total_pop', 'median_age', 'median_income',
    'white_pct', 'tmin_d-5', 'tmax_d-5', 'wspd_d-5', 'tmin_d-4', 'tmax_d-4',
    'wspd_d-4', 'tmin_d-3', 'tmax_d-3', 'wspd_d-3', 'tmin_d-2', 'tmax_d-2',
    'wspd_d-2', 'tmin_d-1', 'tmax_d-1', 'wspd_d-1'
]
x_train = get_correct_types_x(x_train, numeric_cols)
x_valid = get_correct_types_x(x_valid, numeric_cols)
x_test = get_correct_types_x(x_test, numeric_cols)
rel_x_train = x_train[numeric_cols]
rel_x_valid = x_valid[numeric_cols]
rel_x_test = x_test[numeric_cols]

scaler = StandardScaler()
scaler.fit(rel_x_train)
scaled_x_train = scaler.transform(rel_x_train)
scaled_x_valid = scaler.transform(rel_x_valid)
scaled_x_test = scaler.transform(rel_x_test)


In [6]:
class base_model(torch.nn.Module):

    def __init__(self, n_hidden_layers, n_hidden_units, activation=torch.nn.ReLU()):
        super(base_model, self).__init__()
        if n_hidden_layers == 0:
            self.linears =torch.nn.ModuleList([
                torch.nn.Linear(scaled_x_train.shape[1], 1)
            ])
            self.activation = activation
        else:
            assert len(n_hidden_units) == n_hidden_layers
            self.layers = []
            
            for layer, n_units in enumerate(n_hidden_units):
                if layer == 0:
                    curr_layer = torch.nn.Linear(scaled_x_train.shape[1], n_units)
                else:
                    curr_layer = torch.nn.Linear(n_hidden_units[layer - 1], n_units)
                self.layers.append(curr_layer)
            self.layers.append(torch.nn.Linear(n_hidden_units[-1], 1))
            self.linears = torch.nn.ModuleList(self.layers)
            self.activation = activation

    def forward(self, x):
        for layer in self.linears:
            x = self.activation(layer(x))
        return x


In [7]:
# x = torch.from_numpy(scaled_x_train).float()
# y = torch.from_numpy(y_train.values.reshape(-1, 1)).float()

# inputs = Variable(x)
# targets = Variable(y)

# # base = base_model(1, [1], activation=torch.nn.Tanh())
# base = base_model(2, [6, 3])
# print(base)
# optimizer = torch.optim.Adagrad(base.parameters(), lr=0.2)
# loss_func = torch.nn.MSELoss()

# for i in range(100000):
#    prediction = base(inputs)
#    loss = loss_func(prediction, targets)
#    if i % 100 == 0:
#       print(loss)
#    optimizer.zero_grad()
#    loss.backward()
#    optimizer.step()


- Used TanH instead of ReLU
- Adagrad instead of SGD -> SGD just returned 0 for all predictions
- More layers -> more overfitting, run simpler networks for more epochs gets better test error

In [8]:
# pd.Series(prediction.detach().numpy().reshape(-1))

In [9]:
# np.sqrt(loss.detach().numpy())

In [10]:
# test_x = Variable(torch.from_numpy(scaled_x_test).float())
# test_y = Variable(torch.from_numpy(y_test.values.reshape(-1, 1)).float())
# test_predictions = base(test_x)
# valid_x = Variable(torch.from_numpy(scaled_x_valid).float())
# valid_y = Variable(torch.from_numpy(y_valid.values.reshape(-1, 1)).float())
# valid_predictions = base(valid_x)

In [11]:
# loss = loss_func(valid_predictions, valid_y)
# print(np.sqrt(loss.detach().numpy()))


In [12]:
# 1. Define an objective function to be maximized.
def objective(trial):

    # 2. Suggest values of the hyperparameters using a trial object.
    n_layers = trial.suggest_int('n_layers', 0, 3)
    n_hidden_units = [0] * n_layers
    print(n_layers)
    for i in range(n_layers):
        n_hidden_units[i] = trial.suggest_int(f"n_h_{i}", 1, 100)
    lr = trial.suggest_float("lr", 1e-5, 5e-1, log=True)
    n_epochs = trial.suggest_int("n_epochs", 1000, 100000)
    print(f"""Params:
          n_layers: {n_layers}
          n_hidden_units: {n_hidden_units}
          lr: {lr}
          n_epochs: {n_epochs}""")
        
    x = torch.from_numpy(scaled_x_train).float()
    y = torch.from_numpy(y_train.values.reshape(-1, 1)).float()

    inputs = Variable(x)
    targets = Variable(y)

    # base = base_model(1, [1], activation=torch.nn.Tanh())
    base = base_model(n_layers, n_hidden_units)
    optimizer = torch.optim.Adagrad(base.parameters(), lr=lr)
    loss_func = torch.nn.MSELoss()

    for i in range(n_epochs):
        prediction = base(inputs)
        loss = loss_func(prediction, targets)
        if i % 1000 == 0:
            print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
    valid_x = Variable(torch.from_numpy(scaled_x_valid).float())
    valid_y = Variable(torch.from_numpy(y_valid.values.reshape(-1, 1)).float())
    valid_predictions = base(valid_x)
    loss = loss_func(valid_predictions, valid_y)
    print(f"Final valid loss: {loss}")
    print("#################")
    return np.sqrt(loss.detach().numpy())


In [14]:
# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
pd.DataFrame.from_dict({"value": study.best_trial.values, "params": str(
    study.best_trial.params)}).to_csv("nn_hpo/run_1.csv", index=False)

fig = optuna.visualization.plot_optimization_history(study)
# fig.show()
fig.write_image("nn_hpo/run_1.png")


[I 2022-11-25 14:37:51,456] A new study created in memory with name: no-name-b499b87c-2199-4190-81d6-4d240c6b8871


3
Params:
          n_layers: 3
          n_hidden_units: [75, 85, 75]
          lr: 1.637924578685429e-05
          n_epochs: 81348
tensor(9497003., grad_fn=<MseLossBackward0>)
tensor(9496752., grad_fn=<MseLossBackward0>)
tensor(9496636., grad_fn=<MseLossBackward0>)
tensor(9496543., grad_fn=<MseLossBackward0>)
tensor(9496460., grad_fn=<MseLossBackward0>)
tensor(9496385., grad_fn=<MseLossBackward0>)
tensor(9496315., grad_fn=<MseLossBackward0>)
tensor(9496247., grad_fn=<MseLossBackward0>)
tensor(9496181., grad_fn=<MseLossBackward0>)
tensor(9496118., grad_fn=<MseLossBackward0>)
tensor(9496057., grad_fn=<MseLossBackward0>)
tensor(9495995., grad_fn=<MseLossBackward0>)
tensor(9495934., grad_fn=<MseLossBackward0>)
tensor(9495875., grad_fn=<MseLossBackward0>)
tensor(9495818., grad_fn=<MseLossBackward0>)
tensor(9495760., grad_fn=<MseLossBackward0>)
tensor(9495701., grad_fn=<MseLossBackward0>)
tensor(9495644., grad_fn=<MseLossBackward0>)
tensor(9495586., grad_fn=<MseLossBackward0>)
tensor(94955

[I 2022-11-25 14:39:03,927] Trial 0 finished with value: 3056.619873046875 and parameters: {'n_layers': 3, 'n_h_0': 75, 'n_h_1': 85, 'n_h_2': 75, 'lr': 1.637924578685429e-05, 'n_epochs': 81348}. Best is trial 0 with value: 3056.619873046875.


Final valid loss: 9342925.0
#################
0
Params:
          n_layers: 0
          n_hidden_units: []
          lr: 0.03193453545811293
          n_epochs: 97086
tensor(9496054., grad_fn=<MseLossBackward0>)
tensor(9446492., grad_fn=<MseLossBackward0>)
tensor(9425732., grad_fn=<MseLossBackward0>)
tensor(9409883., grad_fn=<MseLossBackward0>)
tensor(9396577., grad_fn=<MseLossBackward0>)
tensor(9384892., grad_fn=<MseLossBackward0>)
tensor(9374359., grad_fn=<MseLossBackward0>)
tensor(9364698., grad_fn=<MseLossBackward0>)
tensor(9355729., grad_fn=<MseLossBackward0>)
tensor(9347324., grad_fn=<MseLossBackward0>)
tensor(9339393., grad_fn=<MseLossBackward0>)
tensor(9331864., grad_fn=<MseLossBackward0>)
tensor(9324683., grad_fn=<MseLossBackward0>)
tensor(9317810., grad_fn=<MseLossBackward0>)
tensor(9311208., grad_fn=<MseLossBackward0>)
tensor(9304850., grad_fn=<MseLossBackward0>)
tensor(9298710., grad_fn=<MseLossBackward0>)
tensor(9292769., grad_fn=<MseLossBackward0>)
tensor(9287009., grad_f

[I 2022-11-25 14:39:11,382] Trial 1 finished with value: 2975.86279296875 and parameters: {'n_layers': 0, 'lr': 0.03193453545811293, 'n_epochs': 97086}. Best is trial 1 with value: 2975.86279296875.


...

tensor(9497148., grad_fn=<MseLossBackward0>)
Final valid loss: 9348601.0
#################
3
Params:
          n_layers: 3
          n_hidden_units: [23, 61, 34]
          lr: 0.00586695277364326
          n_epochs: 88143
tensor(9496862., grad_fn=<MseLossBackward0>)
tensor(1358856.3750, grad_fn=<MseLossBackward0>)
tensor(1091305.7500, grad_fn=<MseLossBackward0>)
tensor(994731.7500, grad_fn=<MseLossBackward0>)
tensor(940896., grad_fn=<MseLossBackward0>)
tensor(902833.3125, grad_fn=<MseLossBackward0>)
tensor(873032.3750, grad_fn=<MseLossBackward0>)
tensor(850200.6875, grad_fn=<MseLossBackward0>)
tensor(831840.3125, grad_fn=<MseLossBackward0>)
tensor(816738.9375, grad_fn=<MseLossBackward0>)
tensor(804435.4375, grad_fn=<MseLossBackward0>)
tensor(793761.8125, grad_fn=<MseLossBackward0>)
tensor(784314.8750, grad_fn=<MseLossBackward0>)
tensor(775704.8125, grad_fn=<MseLossBackward0>)
tensor(767322.3125, grad_fn=<MseLossBackward0>)
tensor(759490.7500, grad_fn=<MseLossBackward0>)
tensor(752236.3

[I 2022-11-25 14:50:17,766] Trial 27 finished with value: 889.8110961914062 and parameters: {'n_layers': 3, 'n_h_0': 23, 'n_h_1': 61, 'n_h_2': 34, 'lr': 0.00586695277364326, 'n_epochs': 88143}. Best is trial 23 with value: 871.9281616210938.


tensor(581514.4375, grad_fn=<MseLossBackward0>)
Final valid loss: 791763.75
#################
1
Params:
          n_layers: 1
          n_hidden_units: [26]
          lr: 0.0017864622819866562
          n_epochs: 90657
tensor(9496664., grad_fn=<MseLossBackward0>)
tensor(9481816., grad_fn=<MseLossBackward0>)
tensor(9469656., grad_fn=<MseLossBackward0>)
tensor(9457017., grad_fn=<MseLossBackward0>)
tensor(9443874., grad_fn=<MseLossBackward0>)
tensor(9430663., grad_fn=<MseLossBackward0>)
tensor(9417455., grad_fn=<MseLossBackward0>)
tensor(9404260., grad_fn=<MseLossBackward0>)
tensor(9391080., grad_fn=<MseLossBackward0>)
tensor(9377916., grad_fn=<MseLossBackward0>)
tensor(9364779., grad_fn=<MseLossBackward0>)
tensor(9351667., grad_fn=<MseLossBackward0>)
tensor(9338570., grad_fn=<MseLossBackward0>)
tensor(9325498., grad_fn=<MseLossBackward0>)
tensor(9312448., grad_fn=<MseLossBackward0>)
tensor(9299418., grad_fn=<MseLossBackward0>)
tensor(9286407., grad_fn=<MseLossBackward0>)
tensor(9273422.,

[I 2022-11-25 14:50:34,158] Trial 28 finished with value: 2861.944580078125 and parameters: {'n_layers': 1, 'n_h_0': 26, 'lr': 0.0017864622819866562, 'n_epochs': 90657}. Best is trial 23 with value: 871.9281616210938.


Final valid loss: 8190726.5
#################
3
Params:
          n_layers: 3
          n_hidden_units: [17, 47, 26]
          lr: 0.037183220524202844
          n_epochs: 81475
tensor(9497147., grad_fn=<MseLossBackward0>)
tensor(704139.6875, grad_fn=<MseLossBackward0>)
tensor(630032.6875, grad_fn=<MseLossBackward0>)
tensor(557363.2500, grad_fn=<MseLossBackward0>)
tensor(460855.5000, grad_fn=<MseLossBackward0>)
tensor(397785.8750, grad_fn=<MseLossBackward0>)
tensor(367869.2188, grad_fn=<MseLossBackward0>)
tensor(347284.3750, grad_fn=<MseLossBackward0>)
tensor(333226.2812, grad_fn=<MseLossBackward0>)
tensor(320805.0938, grad_fn=<MseLossBackward0>)
tensor(311670.6875, grad_fn=<MseLossBackward0>)
tensor(304426.3438, grad_fn=<MseLossBackward0>)
tensor(299748.8125, grad_fn=<MseLossBackward0>)
tensor(295938.1250, grad_fn=<MseLossBackward0>)
tensor(292913.2500, grad_fn=<MseLossBackward0>)
tensor(290486., grad_fn=<MseLossBackward0>)
tensor(288549.6875, grad_fn=<MseLossBackward0>)
tensor(286286

In [16]:
study.best_trial

FrozenTrial(number=5, values=[894.4140014648438], datetime_start=datetime.datetime(2022, 11, 25, 14, 25, 29, 6298), datetime_complete=datetime.datetime(2022, 11, 25, 14, 25, 31, 569139), params={'n_layers': 1, 'n_h_0': 55, 'lr': 0.4052954218938756, 'n_epochs': 9100}, distributions={'n_layers': IntDistribution(high=5, log=False, low=0, step=1), 'n_h_0': IntDistribution(high=100, log=False, low=1, step=1), 'lr': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'n_epochs': IntDistribution(high=10000, log=False, low=1000, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=5, state=TrialState.COMPLETE, value=None)